In [1]:
!pip install faiss-cpu

In [2]:
import os
import faiss
import openai
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from langchain.chains import TransformChain
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain.schema import Document
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.output_parsers.structured import (
    ResponseSchema,
    StructuredOutputParser,
)
from langchain_core.prompts import (
    PromptTemplate,
)
from langchain_core.output_parsers import (
    StrOutputParser,
)

# 환경변수 불러오기
load_dotenv()

False

In [13]:
folder_path = "/Users/bh/중구/음식점"

# 폴더 내 모든 .csv 파일의 경로를 읽어오기
csv_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.csv')]

# 각 CSV 파일을 pandas DataFrame으로 읽어오기
dataframes = []
for file in csv_files:
    df = pd.read_csv(file)
    dataframes.append(df)
    print(f"Loaded {file}, shape: {df.shape}")

# 모든 데이터프레임 리스트 확인
print(f"Total files loaded: {len(dataframes)}")

# 필요시 데이터프레임들을 하나로 합치기 (예: 동일한 형식의 파일이라면)
combined_df1 = pd.concat(dataframes, ignore_index=True)




Loaded /Users/bh/중구/음식점/중구_을지로동_음식점5.csv, shape: (20, 17)
Loaded /Users/bh/중구/음식점/중구_을지로동_음식점4.csv, shape: (70, 17)
Loaded /Users/bh/중구/음식점/중구_명동_음식점4.csv, shape: (70, 17)
Loaded /Users/bh/중구/음식점/중구_명동_음식점5.csv, shape: (20, 17)
Loaded /Users/bh/중구/음식점/중구_을지로동_음식점3.csv, shape: (69, 17)
Loaded /Users/bh/중구/음식점/중구_명동_음식점1.csv, shape: (67, 17)
Loaded /Users/bh/중구/음식점/중구_을지로동_음식점2.csv, shape: (70, 17)
Loaded /Users/bh/중구/음식점/중구_명동_음식점2.csv, shape: (70, 17)
Loaded /Users/bh/중구/음식점/중구_명동_음식점3.csv, shape: (70, 17)
Loaded /Users/bh/중구/음식점/중구_을지로동_음식점1.csv, shape: (68, 17)
Loaded /Users/bh/중구/음식점/중구_중림동_음식점1.csv, shape: (68, 17)
Loaded /Users/bh/중구/음식점/중구_신당5동_음식점1.csv, shape: (68, 17)
Loaded /Users/bh/중구/음식점/중구_신당동_음식점1.csv, shape: (64, 17)
Loaded /Users/bh/중구/음식점/중구_신당동_음식점3.csv, shape: (70, 17)
Loaded /Users/bh/중구/음식점/중구_중림동_음식점2.csv, shape: (70, 17)
Loaded /U

In [14]:
folder_path = "/Users/bh/중구/술집"

# 폴더 내 모든 .csv 파일의 경로를 읽어오기
csv_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.csv')]

# 각 CSV 파일을 pandas DataFrame으로 읽어오기
dataframes = []
for file in csv_files:
    df = pd.read_csv(file)
    dataframes.append(df)
    print(f"Loaded {file}, shape: {df.shape}")

# 모든 데이터프레임 리스트 확인
print(f"Total files loaded: {len(dataframes)}")

# 필요시 데이터프레임들을 하나로 합치기 (예: 동일한 형식의 파일이라면)
combined_df2 = pd.concat(dataframes, ignore_index=True)




Loaded /Users/bh/중구/술집/중구_신당5동_술집4.csv, shape: (70, 17)
Loaded /Users/bh/중구/술집/중구_중림동_술집2.csv, shape: (60, 17)
Loaded /Users/bh/중구/술집/중구_약수동_술집1.csv, shape: (68, 17)
Loaded /Users/bh/중구/술집/중구_중림동_술집3.csv, shape: (60, 17)
Loaded /Users/bh/중구/술집/중구_신당5동_술집5.csv, shape: (20, 17)
Loaded /Users/bh/중구/술집/중구_광희동_술집4.csv, shape: (70, 17)
Loaded /Users/bh/중구/술집/중구_중림동_술집1.csv, shape: (68, 17)
Loaded /Users/bh/중구/술집/중구_약수동_술집2.csv, shape: (70, 17)
Loaded /Users/bh/중구/술집/중구_약수동_술집3.csv, shape: (70, 17)
Loaded /Users/bh/중구/술집/중구_광희동_술집5.csv, shape: (20, 17)
Loaded /Users/bh/중구/술집/중구_신당5동_술집2.csv, shape: (70, 17)
Loaded /Users/bh/중구/술집/중구_광희동_술집1.csv, shape: (67, 17)
Loaded /Users/bh/중구/술집/중구_중림동_술집4.csv, shape: (59, 17)
Loaded /Users/bh/중구/술집/중구_중림동_술집5.csv, shape: (20, 17)
Loaded /Users/bh/중구/술집/중구_신당5동_술집3.csv, shape: (69, 17)
Loaded /Users/bh/중구/술집/중구_신당5동_술집1.csv, shape: (68, 17)
Loa

In [15]:
folder_path = "/Users/bh/중구/카페"

# 폴더 내 모든 .csv 파일의 경로를 읽어오기
csv_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.csv')]

# 각 CSV 파일을 pandas DataFrame으로 읽어오기
dataframes = []
for file in csv_files:
    df = pd.read_csv(file)
    dataframes.append(df)
    print(f"Loaded {file}, shape: {df.shape}")

# 모든 데이터프레임 리스트 확인
print(f"Total files loaded: {len(dataframes)}")

# 필요시 데이터프레임들을 하나로 합치기 (예: 동일한 형식의 파일이라면)
combined_df3 = pd.concat(dataframes, ignore_index=True)




Loaded /Users/bh/중구/카페/중구_청구동_카페3.csv, shape: (70, 17)
Loaded /Users/bh/중구/카페/중구_장충동_카페2.csv, shape: (70, 17)
Loaded /Users/bh/중구/카페/중구_다산동_카페3.csv, shape: (70, 17)
Loaded /Users/bh/중구/카페/중구_신당동_카페4.csv, shape: (70, 17)
Loaded /Users/bh/중구/카페/중구_신당동_카페5.csv, shape: (20, 17)
Loaded /Users/bh/중구/카페/중구_다산동_카페2.csv, shape: (70, 17)
Loaded /Users/bh/중구/카페/중구_장충동_카페3.csv, shape: (70, 17)
Loaded /Users/bh/중구/카페/중구_청구동_카페2.csv, shape: (70, 17)
Loaded /Users/bh/중구/카페/중구_황학동_카페5.csv, shape: (20, 17)
Loaded /Users/bh/중구/카페/중구_장충동_카페1.csv, shape: (64, 17)
Loaded /Users/bh/중구/카페/중구_다산동_카페1.csv, shape: (68, 17)
Loaded /Users/bh/중구/카페/중구_황학동_카페4.csv, shape: (70, 17)
Loaded /Users/bh/중구/카페/중구_청구동_카페1.csv, shape: (68, 17)
Loaded /Users/bh/중구/카페/중구_청구동_카페5.csv, shape: (20, 17)
Loaded /Users/bh/중구/카페/중구_장충동_카페4.csv, shape: (70, 17)
Loaded /Users/bh/중구/카페/중구_다산동_카페5.csv, shape: (20, 17)
Loaded /Users/bh/중구/카페/중구_신당동_카페2.c

In [17]:
data_jg = pd.concat([combined_df1,combined_df2,combined_df3])

In [24]:
data_jg = data_jg.drop_duplicates()

In [26]:
data_jg.rename(columns={'Unnamed: 0': 'store_name'}, inplace=True)

/var/folders/_v/g1qmsq595kq_fg_zv8n2022h0000gn/T/ipykernel_57932/2590068818.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_jg.rename(columns={'Unnamed: 0': 'store_name'}, inplace=True)


In [28]:
data_jg.to_csv('jg.csv')

In [15]:
data = pd.read_csv('jg.csv', encoding='utf-8')
data.iloc[3]

store_name                                                                웨스틴 조선 서울 조선델리
category                                                                            베이커리
new_open                                                                                
rating                                                                           별점 4.57
visited_review                                                                방문자 리뷰 754
directions_text                                                                         
store_id                                                                       558674555
address                                             서울 중구 소공로 106 지하 1층, 웨스틴 조선 서울 저층 로비
blog_review                                                                   블로그 리뷰 436
phone_num                                                                    02-317-0022
business_hours                         목\r\n07:00 - 21:00\r\n금\r\n07:00 - 21:00\r\n토\...
info                 

In [16]:
def split_into_domains(document):
    # Extract metadata and content
    metadata = document.metadata  # Use direct attribute access
    content = document.page_content  # Use direct attribute access

    # Parse content into structured data
    structured_data = {}
    for line in content.split("\n"):
        if ":" in line:
            key, value = line.split(":", 1)
            structured_data[key.strip()] = value.strip()

    # Create domain-specific metadata
    domains = {
        "store_info": {
            "store_name": structured_data.get("store_name"),
            "category": structured_data.get("category", "").split(","),
            "rating": structured_data.get("rating"),
            "address": structured_data.get("address"),
            "phone_num": structured_data.get("phone_num"),
            "sns": structured_data.get("sns"),
        },
        "business_info": {
            "business_hours": {
                k.strip(): v.strip()
                for k, v in zip(
                    structured_data.get("business_hours", "").split("\n")[::2],
                    structured_data.get("business_hours", "").split("\n")[1::2],
                )
            },
            "parking": structured_data.get("parking"),
            "convenience_facilities_and_services": structured_data.get(
                "convenience_facilities_and_services", ""
            ).split("\n"),
        },
        "review_info": {
            "visited_review": structured_data.get("visited_review"),
            "blog_review": structured_data.get("blog_review"),
            "review_like_this_part_output": structured_data.get("blog_review"),
        },
    }

    return domains

In [18]:
# CSVLoader 클래스를 사용하여 CSV 파일 로드
loader = CSVLoader(file_path='./중구_소공동_카페1.csv')  # CSV 파일 경로 지정

# CSV 파일에서 데이터 내용 로드
loaded_documents = loader.load()  # 로드된 데이터가 loaded_documents 변수에 저장

# Split documents into domain-specific data
domain_documents = [split_into_domains(document) for document in loaded_documents]

# Combine all domains into a single metadata dictionary
processed_documents = [
    {
        "page_content": "\n".join(
            f"{key}: {value}" for domain in document.values() for key, value in domain.items() if value
        ),
        "metadata": {
            domain_name: domain
            for domain_name, domain in document.items()
        },  # Merge all domain data into metadata
    }
    for document in domain_documents
]

# Embedding and FAISS indexing
model_embedding = OpenAIEmbeddings(model='text-embedding-ada-002')
db = FAISS.from_documents(
    [Document(page_content=d["page_content"], metadata=d["metadata"]) for d in processed_documents],
    embedding=model_embedding,
)

# Save FAISS index
db.save_local("faiss_index_jg")

In [37]:
retriever = db.as_retriever(
    searcy_type='mmr',
    search_kwargs={
        'fetch_k': 20,        # mmr에 전달할 문서 수
        'k': 6,               # 반환 문서 수
        'lambda_mult': 0.25,  # 다양성 조정 파라미터
        # 'filter': {'source': './data/keywords.txt'}        # 메타데이터 기반 필터링
    }
)
answer = retriever.invoke('중구에 있는 리뷰 많은 술집 하나 추천해줘')
print(answer[0].page_content)

store_name: 크로플각 중구점
category: ['카페', '디저트']
rating: 정보없음
address: 서울 중구 장충단로 191


In [42]:
# question =  '''
#             서울에 3박 4일동안 여행을 갈거야. 여행의 목적은 한국의 K-POP 관련 관광지를 방문하는 것이야. 
#             추천해줄 수 있어?
#             '''

question = '중구에서 여행을 갈거야 을지로동에 전주옥을 포함해서 일정을 만들어줘'

template = '''
    너는 벡터 DB의 데이터만 사용해서 서울의 관광 일정을 추천해주는 봇이야.
    사용자의 질문에 따라서 여러가지 서울의 관광지, 식당, 숙소, 쇼핑몰 등을 추천해야해.
    처음에 사용자가 서울에 며칠동안 머무르는지 물어보고, 그에 따라서 관광 일정을 추천해주면 돼.
    그 다음 사용자가 서울에 오는 이유에 대해 파악하고 그에 맞는 여행 일정을 추천해주면 돼.
    대화하다가 추천한 여행 일정이 사용자의 마음에 들지 않아서 변경해달라고 하면 마음에 들지 않은 부분을
    캐치하고 그 부분들만 수정해서 다시 추천해주면 돼.
    일정 중간 중간 숙소나 식당, 쇼핑몰 등을 추천해주면 돼.

    일정에 대해 서로 대화하다가 어떤 장소가 궁금해서 물어본다면 특정 장소의 정보들을 정확하게 알려줘.
    (예:가게 정보, 메뉴, 별점, 위치, 연락처, 운영 시간, 리뷰 등)
    여행 일정에 대한 동선이 웬만하면 짧도록 추천해줘.
    언어는 사용자가 입력한 언어를 기준으로 알려줘. 
    화폐 기준도 사용자가 입력한 언어를 사용하는 국가의 화폐를 기준으로 적용해줘.

    질문: {question}

    내용: {reference}

    언어: {language}
'''

prompt = PromptTemplate(
    template=template,
    input_variables=['question', 'reference', 'language'],
)

model = ChatOpenAI(model='gpt-4o-mini')
parser = StrOutputParser()
chain = (
    {
        'reference': itemgetter('question') | retriever,
        'question': itemgetter('question'),
        'language': itemgetter('language'),
    }
    | prompt
    | model
    | parser
)

# chain을 통해 question
reference = chain.invoke(
    {
        'question': f'{question}',
        'language': '한국어',
    }
)

print(f"Answer: \n{reference}")

Answer: 
서울 중구에서의 여행 일정을 아래와 같이 추천드립니다. 을지로동에 위치한 전주옥을 포함하여 다양한 관광지와 식당을 포함했습니다.

### 첫째 날
1. **전주옥** (점심)
   - **종류**: 육류, 고기요리
   - **별점**: 4.43
   - **주소**: 서울 중구 을지로 101 1층
   - **전화번호**: 02-2279-1710

2. **을지로 오뎅바** (간식)
   - **종류**: 오뎅, 꼬치
   - **주소**: 서울 중구 을지로12길 14

3. **금옥당 서울역점** (디저트)
   - **종류**: 카페, 디저트
   - **주소**: 서울 중구 한강대로 405 1층
   - **전화번호**: 0507-1336-6332
   - [웹사이트](http://guemokdang.com/)

4. **신당동 떡볶이 골목** (저녁)
   - 다양한 간식과 저녁 메뉴를 즐길 수 있는 곳입니다.

### 둘째 날
1. **남산타워** (관광)
   - 서울의 아름다운 전망을 감상할 수 있는 명소입니다.

2. **명동 쇼핑 거리** (쇼핑)
   - 다양한 쇼핑과 먹거리를 즐길 수 있는 거리입니다.

3. **저녁: 펀비어킹 중구 신당점**
   - **종류**: 맥주, 호프
   - **주소**: 서울 중구 퇴계로76길 32 2층
   - **전화번호**: 02-2252-3378

이 일정이 마음에 드시나요? 혹시 더 추가하고 싶은 장소나 수정하고 싶은 부분이 있으면 말씀해 주세요!
